In [66]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso




In [67]:
df1 = pd.read_csv("/Users/ethanboroditsky/ML_Focus/class_final_proportion.csv")
print (df1.iloc[:3])
# and get summary stats on variables
print (df1.describe())

   party  seats sq_cabinet  sq_pm  election_year  base  miw_new   banzhaf  \
0     MP     16          0    0.0           1998    16        1  0.054545   
1     SJ     25          1    1.0           2007    25        2  0.500000   
2  GRENG      5          0    0.0           1994     5        1  0.076923   

    shapley     splus country  cabinet_id  party_id cabinet_name  caretaker  \
0  0.052381  0.037114     SWE         503      1154   Persson II          0   
1  0.500000  0.458311     ISL         558      1342    Haarde II          0   
2  0.066667  0.051242     LUX         222       310   Santer III          0   

   cabinet_party  prime_minister  left_rightx  left_righty  cabinet_seats  \
0              0               0     3.373333          NaN              0   
1              1               1     6.944444          NaN              6   
2              0               0     1.578947          NaN              0   

   total_cabinet_size  party_count  cab_count  \
0               

In [68]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [69]:
df1 = swap_columns(df1, "coalition_total", "cabinet_proportion")


In [70]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [71]:
df1.corr()

,seats,sq_pm,election_year,base,miw_new,banzhaf,shapley,splus,cabinet_id,party_id,caretaker,cabinet_party,prime_minister,left_rightx,left_righty,cabinet_seats,total_cabinet_size,party_count,cab_count,country_id,election_id,seats_share,post_election,enpp,mingov,bicameral,largest_parl,largest_cab,lag_largest_parl,lag_largest_cab,seats_total,miw_proportion,coalition_total,seats_proportion,W,A,B,B_star,C,D,E,country_dummy1,country_dummy2,country_dummy3,country_dummy4,country_dummy5,country_dummy6,country_dummy7,country_dummy8,country_dummy9,country_dummy10,country_dummy11,country_dummy12,country_dummy13,cabinet_proportion
seats,1.000000,0.400724,-0.007225,1.000000,0.377642,0.590770,0.606373,0.612598,0.040589,-0.027959,-0.019316,0.274634,0.433421,0.019041,-0.105539,0.518469,0.104983,-0.085618,-0.002844,0.086471,0.019188,0.677957,0.009861,-0.182858,-0.042072,0.124159,0.468271,0.451649,0.388166,0.366939,0.430233,0.614627,0.526376,0.700943,0.430176,NaN,0.083511,0.169618,0.084384,0.022700,-0.149897,0.044224,-0.137346,0.491313,-0.067968,-0.060805,-0.065319,-0.109429,0.131415,-0.100746,-0.124630,-0.057982,0.038978,0.108629,0.490097
sq_pm,0.400724,1.000000,0.006623,0.400724,0.120896,0.586115,0.595622,0.600797,0.009085,-0.024205,0.015155,0.259422,0.529755,0.048575,-0.051598,0.439140,-0.061930,-0.130514,-0.073925,0.031458,-0.002880,0.595999,-0.009999,-0.095270,-0.004451,-0.041483,0.625554,0.566529,0.737216,0.792539,-0.034660,0.565516,-0.060132,0.599507,-0.034730,NaN,0.060052,0.041349,0.061330,-0.022728,-0.072494,0.065050,-0.044945,0.057734,-0.003575,-0.011685,0.005071,0.041349,-0.076019,0.028212,-0.028361,0.003641,0.029712,0.011969,0.468018
election_year,-0.007225,0.006623,1.000000,-0.007225,-0.056797,0.019711,0.020308,0.020764,0.252269,0.011598,-0.019712,0.032109,0.001360,-0.023947,0.002113,-0.000927,-0.163253,-0.113730,-0.059413,-0.009615,-0.033121,0.024627,0.013737,0.065129,0.010498,-0.029827,0.011499,0.011870,0.003448,0.007222,-0.081474,0.026825,-0.116300,0.023474,-0.081203,NaN,-0.150477,-0.206525,-0.073633,0.181363,0.024021,-0.063606,0.022304,0.003688,-0.042825,0.009832,-0.023249,-0.010460,-0.089686,0.010260,0.091881,0.020821,0.073462,0.022222,0.012791
base,1.000000,0.400724,-0.007225,1.000000,0.377642,0.590770,0.606373,0.612598,0.040589,-0.027959,-0.019316,0.274634,0.433421,0.019041,-0.105539,0.518469,0.104983,-0.085618,-0.002844,0.086471,0.019188,0.677957,0.009861,-0.182858,-0.042072,0.124159,0.468271,0.451649,0.388166,0.366939,0.430233,0.614627,0.526376,0.700943,0.430176,NaN,0.083511,0.169618,0.084384,0.022700,-0.149897,0.044224,-0.137346,0.491313,-0.067968,-0.060805,-0.065319,-0.109429,0.131415,-0.100746,-0.124630,-0.057982,0.038978,0.108629,0.490097
miw_new,0.377642,0.120896,-0.056797,0.377642,1.000000,0.170115,0.184195,0.184913,-0.030808,0.066959,0.016576,0.191185,0.144233,0.025843,-0.049090,0.217084,0.360849,0.489165,0.343250,-0.089844,-0.057597,0.174434,0.043544,0.292803,-0.111790,0.274405,0.199992,0.188402,0.119024,0.105943,0.339799,0.168283,0.372997,0.168930,0.340022,NaN,-0.206733,-0.098652,-0.100058,0.003641,0.263735,-0.095829,0.104298,-0.096681,-0.059146,-0.073971,-0.112256,-0.096896,0.470826,-0.073481,0.034595,-0.095733,-0.068834,-0.093435,0.148894
banzhaf,0.590770,0.586115,0.019711,0.590770,0.170115,1.000000,0.995847,0.991502,0.006343,-0.018308,-0.055107,0.456485,0.661531,0.073890,-0.110753,0.723964,-0.142569,-0.276987,-0.166169,0.069695,0.004688,0.890178,-0.022191,-0.218308,-0.012684,-0.091168,0.728791,0.682675,0.616783,0.529142,-0.067946,0.957035,-0.089136,0.891292,-0.068135,NaN,0.131222,0.094027,0.139911,-0.029879,-0.177226,0.148456,-0.104096,0.131653,-0.009785,-0.027755,0.005627,0.094027,-0.152219,0.064021,-0.066036,0.007510,0.067558,0.026557,0.759571
shapley,0.606373,0.595622,0.020308,0.606373,0.184195,0.995847,1.000000,0.994279,0.006590,-0.014620,-0.056577,0.462564,0.672913,0.081262,-0.110776,0.726586,-0.146341,-0.284381,-0.170632,0.071517,0.004740,0.906661,-0.022769,-0.224159,-0.012998,-0.093610,0.735864,0.694490,0

In [72]:
df1.fillna((df1.median()+df1.mean())/2, inplace=True)
df1.dropna()

,party,seats,sq_cabinet,sq_pm,election_year,base,miw_new,banzhaf,shapley,splus,country,cabinet_id,party_id,cabinet_name,caretaker,cabinet_party,prime_minister,left_rightx,left_righty,cabinet_seats,total_cabinet_size,party_count,cab_count,party_name,party_name_english,country_id,election_id,seats_share,election_date,start_date,post_election,enpp,mingov,bicameral,largest_parl,largest_cab,lag_largest_parl,lag_largest_cab,seats_total,miw_proportion,coalition_total,seats_proportion,W,A,B,B_star,C,D,E,country_dummy1,country_dummy2,country_dummy3,country_dummy4,country_dummy5,country_dummy6,country_dummy7,country_dummy8,country_dummy9,country_dummy10,country_dummy11,country_dummy12,country_dummy13,cabinet_proportion
0,MP,16,0,0.000000,1998,16,1,0.054545,0.052381,0.037114,SWE,503,1154,Persson II,0,0,0,3.373333,5.561957,0,22,7,1,Miljæpartiet de Græna,Greens,35,33,4.584527,9/21/1998,10/7/1998,1,4.293757,1,0,0,0,0,0,349,0.066667,130.366953,0.045845,175,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000000
1,SJ,25,1,1.000000,2007,25,2,0.500000,0.500000,0.458311,ISL,558,1342,Haarde II,0,1,1,6.944444,5.561957,6,12,5,2,Sj_lfst__isflokkurinn,Independence Party,56,265,39.682541,5/12/2007,5/24/2007,1,3.624658,0,0,1,1,1,1,63,0.400000,43.000000,0.396825,32,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.500000
2,GRENG,5,0,0.000000,1994,5,1,0.076923,0.066667,0.051242,LUX,222,310,Santer III,0,0,0,1.578947,5.561957,0,15,5,2,D_i Gr_ng _ Les Verts _ Die Grnen,The Greens,7,306,8.333333,6/12/1994,7/13/1994,1,3.896104,0,0,0,0,0,0,60,0.111111,130.366953,0.083333,31,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.000000
3,GRAEN,6,0,0.000000,1999,6,1,0.166667,0.166667,0.121258,ISL,177,210,Oddsson III,0,0,0,1.447368,5.561957,0,12,5,2,Vinstrihreyfingin _ gr_nt frambo_,Left-Green Movement,56,592,9.523809,5/8/1999,5/28/1999,1,3.454308,0,0,0,0,0,0,63,0.200000,130.366953,0.095238,32,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000000
4,CSV,21,1,1.000000,1994,21,3,0.384615,0.400000,0.379442,LUX,222,1234,Santer III,0,1,1,6.710526,5.561957,8,15,5,2,Chrschtlech Sozial Vollekspartei _ Parti popu...,Christian Social People's Party,7,306,35.000000,6/12/1994,7/13/1994,1,3.896104,0,0,1,1,1,1,60,0.333333,38.000000,0.350000,31,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.533333
5,CDU/CSU,245,1,1.000000,1998,245,1,0.166667,0.166667,0.144816,DEU,107,808,Schroeder I,0,0,0,6.175778,6.700000,0,17,6,2,Christlich Demokratische Union,Christian Democratic Union,54,679,29.596413,9/27/1998,10/27/1998,1,3.301303,0,1,0,0,0,1,669,0.200000,130.366953,0.366218,335,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.000000
6,GRUE,55,1,0.000000,2002,55,1,0.333333,0.333333,0.252555,DEU,529,772,Schroeder II,-9999,1,0,2.874444,2.800000,3,14,6,2,BÎÙndnis 90 / Die GrÎÙnen,Alliance 90 / Greens,54,385,9.121061,9/22/2002,10/22/2002,1,3.376034,0,1,0,0,0,0,603,0.333333,306.000000,0.091211,302,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.214286
7,GRENG,7,0,0.000000,2004,7,1,0.166667,0.166667,0.120291,LUX,12,310,Juncker III,0,0,0,1.578947,5.561957,0,25,5,2,D_i Gr_ng _ Les Verts _ Die Grnen,The Greens,7,383,11.666667,6/13/2004,7/31/2004,1,3.805497,0,0,0,0,0,0,60,0.200000,130.366953,0.116667,31,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.000000
8,OVP,52,1,0.000000,1994,52,1,0.333333,0.333333,0.314240,AUT,294,1013,Vranitzky IV,0,1,0,6.366667,5.800000,9,17,5,2,sterreichische Volkspartei,Austrian People's Party,59,66,28.415300,10/9/1994,11/29/1994,1,3.728042,0,1,0,0,0,0,183,0.333333,117.000000,0.284153,92,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.562500
9,FR,22,0,0.000000,1989,22,2,0.153846,0.166667,0.148417,NOR,94,351,Syse,0,0,0,9.244444,9.400000,0,19,7,3,Fremskrittspartiet,Progress Party,9,571,13.333333,9/11/1989,10/16/1989,1,4.234718,1,0,0,0,0,0,165,0.181818,130.366953,0.133333,83,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.000000


In [73]:
#dim = 1
power = ["banzhaf", "shapley", "splus", "miw_proportion", "seats_share"]
largest = ["largest_parl", "largest_cab"]
largest_lag = ["lag_largest_parl", "lag_largest_cab"]
frag = ["enpp","country_dummy2", "party_count", "cab_count"]

#raw variables: PM, cabinet parties, seat

x_norm = StandardScaler().fit_transform(df1.loc[:, frag])


pca = PCA(n_components=1).fit(x_norm)
new_df = pca.transform(x_norm)
var_exp = pca.explained_variance_ratio_

print ("Variance explained by each latent variable in PCA: ", pca.explained_variance_ratio_ )


Variance explained by each latent variable in PCA:  [0.62111627]


In [88]:
x_norm_p = StandardScaler().fit_transform(df1.loc[:, power])
x_norm_l = StandardScaler().fit_transform(df1.loc[:, largest])
x_norm_ll = StandardScaler().fit_transform(df1.loc[:, largest_lag])


pcap = PCA(n_components=1).fit(x_norm_p)
pcal = PCA(n_components=1).fit(x_norm_l)
pcall = PCA(n_components=1).fit(x_norm_ll)

np1 = pcap.transform(x_norm_p)
np2 = pcal.transform(x_norm_l)
np3 = pcall.transform(x_norm_ll)

df1["pcap"] = np1[:,0]
df1["pcal"] = np2[:,0]
df1["pcall"] = np3[:,0]

x_norm_f = StandardScaler().fit_transform(df1.loc[:, frag])
pcaf = PCA(n_components=1).fit(x_norm_f)
np4 = pcaf.transform(x_norm_f)
df1["pcaf"] = np4[:,0]









df1.head()

,party,seats,sq_cabinet,sq_pm,election_year,base,miw_new,banzhaf,shapley,splus,country,cabinet_id,party_id,cabinet_name,caretaker,cabinet_party,prime_minister,left_rightx,left_righty,cabinet_seats,total_cabinet_size,party_count,cab_count,party_name,party_name_english,country_id,election_id,seats_share,election_date,start_date,post_election,enpp,mingov,bicameral,largest_parl,largest_cab,lag_largest_parl,lag_largest_cab,seats_total,miw_proportion,coalition_total,seats_proportion,W,A,B,B_star,C,D,E,country_dummy1,country_dummy2,country_dummy3,country_dummy4,country_dummy5,country_dummy6,country_dummy7,country_dummy8,country_dummy9,country_dummy10,country_dummy11,country_dummy12,country_dummy13,cabinet_proportion,pcap,pcal,pcall,pcaf
0,MP,16,0,0.0,1998,16,1,0.054545,0.052381,0.037114,SWE,503,1154,Persson II,0,0,0,3.373333,5.561957,0,22,7,1,Miljæpartiet de Græna,Greens,35,33,4.584527,9/21/1998,10/7/1998,1,4.293757,1,0,0,0,0,0,349,0.066667,130.366953,0.045845,175,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,-1.330700,-0.550673,-0.537772,-1.344910
1,SJ,25,1,1.0,2007,25,2,0.500000,0.500000,0.458311,ISL,558,1342,Haarde II,0,1,1,6.944444,5.561957,6,12,5,2,Sj_lfst__isflokkurinn,Independence Party,56,265,39.682541,5/12/2007,5/24/2007,1,3.624658,0,0,1,1,1,1,63,0.400000,43.000000,0.396825,32,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.500000,5.486999,3.631957,3.719049,-1.562433
2,GRENG,5,0,0.0,1994,5,1,0.076923,0.066667,0.051242,LUX,222,310,Santer III,0,0,0,1.578947,5.561957,0,15,5,2,D_i Gr_ng _ Les Verts _ Die Grnen,The Greens,7,306,8.333333,6/12/1994,7/13/1994,1,3.896104,0,0,0,0,0,0,60,0.111111,130.366953,0.083333,31,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.000000,-0.851937,-0.550673,-0.537772,-1.462405
3,GRAEN,6,0,0.0,1999,6,1,0.166667,0.166667,0.121258,ISL,177,210,Oddsson III,0,0,0,1.447368,5.561957,0,12,5,2,Vinstrihreyfingin _ gr_nt frambo_,Left-Green Movement,56,592,9.523809,5/8/1999,5/28/1999,1,3.454308,0,0,0,0,0,0,63,0.200000,130.366953,0.095238,32,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000000,0.386126,-0.550673,-0.537772,-1.625207
4,CSV,21,1,1.0,1994,21,3,0.384615,0.400000,0.379442,LUX,222,1234,Santer III,0,1,1,6.710526,5.561957,8,15,5,2,Chrschtlech Sozial Vollekspartei _ Parti popu...,Christian Social People's Party,7,306,35.000000,6/12/1994,7/13/1994,1,3.896104,0,0,1,1,1,1,60,0.333333,38.000000,0.350000,31,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.533333,4.108260,3.631957,3.719049,-1.462405


In [91]:
IVs = [ "pcaf", "pcal", 'pcall', "pcap", "prime_minister", "seats", "cabinet_party"]

# create train / test split using dataframe
x_train, x_test, y_train, y_test = train_test_split(df1.loc[:, IVs], df1.loc[:, 'cabinet_proportion'], test_size=0.25, random_state=13)

# make sure results make sense
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)
print("-------")


# try both linear and polynomial of different degrees
linear_model = LinearRegression(normalize=True)
p2_model = LinearRegression(normalize=True)
p3_model = LinearRegression(normalize=True)

# create polynomial features
p2_features = PolynomialFeatures(degree=2)
p2_train = p2_features.fit_transform(x_train)
p2_test = p2_features.fit_transform(x_test)

p3_features = PolynomialFeatures(degree=3)
p3_train = p3_features.fit_transform(x_train)
p3_test = p3_features.fit_transform(x_test)

# now do estimation of models
lin_1 = linear_model.fit(x_train, y_train)
p2_1 = p2_model.fit(p2_train, y_train)
p3_1 = p3_model.fit(p3_train, y_train)

# predict values for test sets
lin1_predict = lin_1.predict(x_test)
p2_predict = p2_1.predict(p2_test)
p3_predict = p3_1.predict(p3_test)

print(mean_squared_error(lin1_predict, y_test))
print(mean_squared_error(p2_predict, y_test))
print(mean_squared_error(p3_predict, y_test))

# use k-fold with regularization
# remember: alpha here is lambda in most treatments
from sklearn.linear_model import LassoCV

#Lasso Cross validation
lasso_1 = LassoCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 10], random_state=0).fit(x_train, y_train)

# results
print(lasso_1.score(x_train, y_train))
print(lasso_1.score(x_test, y_test))
print("----------------")



optimal_alpha = lasso_1.alpha_
print(f"Optimal alpha: {optimal_alpha}")

# 2. Get the model's coefficients
coefficients = lasso_1.coef_
print(coefficients)

# 3. Identify which features were included and which were excluded
# You can get the names of the features and filter them based on non-zero coefficients.
included_features = [feature for feature, coef in zip(x_train.columns, coefficients) if coef != 0]
excluded_features = [feature for feature, coef in zip(x_train.columns, coefficients) if coef == 0]

# Print the results
print(f"Included features: {included_features}")
print(f"Excluded features: {excluded_features}")

(492, 7) (492,)
(165, 7) (165,)
-------
0.008440698526374671
0.00337325885239901
0.16301690316401785
0.8385821985435793
0.861556003990982
----------------
Optimal alpha: 0.0001
[-0.00679486  0.00408879 -0.00261165  0.02926832  0.17344004  0.00025107
  0.22795266]
Included features: ['pcaf', 'pcal', 'pcall', 'pcap', 'prime_minister', 'seats', 'cabinet_party']
Excluded features: []


In [94]:
feature_names = df1.loc[:, IVs].columns.tolist()
linear_coefficients = lin_1.coef_
linear_intercept = lin_1.intercept_
linear_equation = f"y = {linear_intercept:.4f} + " + " + ".join([f"{coef:.4f}*{feature_names[i]}" for i, coef in enumerate(linear_coefficients)])
print("Linear Regression Equation:")
print(linear_equation)

Linear Regression Equation:
y = 0.0193 + -0.0068*pcaf + 0.0037*pcal + -0.0028*pcall + 0.0293*pcap + 0.1760*prime_minister + 0.0002*seats + 0.2282*cabinet_party


In [96]:
# Print polynomial regression equations
def print_polynomial_equation(coefficients, intercept, feature_names, degree):
    terms = []
    for i, coef in enumerate(coefficients):
        if i > 0:  # Skip the intercept
            # Use the feature names for polynomial features
            terms.append(f"{coef:.4f}*{feature_names[i]}")  # Use feature_names directly without adjusting index
    equation = f"y = {intercept:.4f} + " + " + ".join(terms)
    print(f"Polynomial Regression (Degree {degree}) Equation:")
    print(equation)

# For Polynomial Regression (Degree 2)
p2_features = PolynomialFeatures(degree=2)
p2_train = p2_features.fit_transform(x_train)  # Fit to the training data
p2_feature_names = p2_features.get_feature_names(feature_names)
p2_intercept = p2_1.intercept_
p2_coefficients = p2_1.coef_
# Get new feature names

print_polynomial_equation(p2_coefficients, p2_intercept, p2_feature_names, 2)

Polynomial Regression (Degree 2) Equation:
y = -0.0138 + 0.0058*pcaf + -0.0243*pcal + -0.0112*pcall + 0.0028*pcap + 0.0270*prime_minister + 0.0006*seats + 0.1522*cabinet_party + -0.0001*pcaf^2 + 0.0018*pcaf pcal + -0.0029*pcaf pcall + 0.0023*pcaf pcap + 0.0014*pcaf prime_minister + -0.0001*pcaf seats + -0.0201*pcaf cabinet_party + -0.0168*pcal^2 + -0.0075*pcal pcall + 0.0059*pcal pcap + -0.0154*pcal prime_minister + 0.0003*pcal seats + 0.0475*pcal cabinet_party + 0.0022*pcall^2 + 0.0034*pcall pcap + 0.0100*pcall prime_minister + -0.0001*pcall seats + 0.0186*pcall cabinet_party + -0.0016*pcap^2 + -0.0339*pcap prime_minister + 0.0001*pcap seats + 0.0795*pcap cabinet_party + 0.0270*prime_minister^2 + 0.0009*prime_minister seats + 0.0270*prime_minister cabinet_party + -0.0000*seats^2 + -0.0009*seats cabinet_party + 0.1522*cabinet_party^2


In [97]:
# For Polynomial Regression (Degree 3)
p3_features = PolynomialFeatures(degree=3)
p3_train = p3_features.fit_transform(x_train)  # Fit to the training data
p3_feature_names = p3_features.get_feature_names(feature_names)  # Get new feature names
p3_intercept = p3_1.intercept_
p3_coefficients = p3_1.coef_

print_polynomial_equation(p3_coefficients, p3_intercept, p3_feature_names, 3)

Polynomial Regression (Degree 3) Equation:
y = 48410195799618.0781 + -0.1357*pcaf + -7878543181.9281*pcal + -24130684526.8781*pcall + 0.1069*pcap + 8691737896.2256*prime_minister + 0.0009*seats + 6676960051.9256*cabinet_party + 0.0360*pcaf^2 + -0.3239*pcaf pcal + 0.0302*pcaf pcall + 0.0311*pcaf pcap + -1302377779.3734*pcaf prime_minister + 0.0017*pcaf seats + 502627415.5210*pcaf cabinet_party + 4546325792.5595*pcal^2 + -0.1800*pcal pcall + 0.1564*pcal pcap + 1795019582.0970*pcal prime_minister + -0.0015*pcal seats + 7833100771.9610*pcal cabinet_party + 37627364071.0371*pcall^2 + -0.0835*pcall pcap + -2210550961.5958*pcall prime_minister + 0.0034*pcall seats + -677208345.8740*pcall cabinet_party + 0.0205*pcap^2 + 299964021.7317*pcap prime_minister + -0.0010*pcap seats + -43478877.1959*pcap cabinet_party + 6727638287.7812*prime_minister^2 + 7934444.3970*prime_minister seats + 7714068218.9437*prime_minister cabinet_party + 0.0000*seats^2 + -49049585.9724*seats cabinet_party + 2864002134.1

In [98]:
lin_predictions = lin_1.predict(x_test)
lin_mse = mean_squared_error(y_test, lin_predictions)
print(f"Linear Regression MSE: {lin_mse:.4f}")

# For Polynomial Regression (Degree 2)
p2_predictions = p2_model.predict(p2_features.transform(x_test))  # Transform x_test for polynomial features
p2_mse = mean_squared_error(y_test, p2_predictions)
print(f"Polynomial Regression (Degree 2) MSE: {p2_mse:.4f}")

# For Polynomial Regression (Degree 3)
p3_predictions = p3_model.predict(p3_features.transform(x_test))  # Transform x_test for polynomial features
p3_mse = mean_squared_error(y_test, p3_predictions)
print(f"Polynomial Regression (Degree 3) MSE: {p3_mse:.4f}")

Linear Regression MSE: 0.0084
Polynomial Regression (Degree 2) MSE: 0.0034
Polynomial Regression (Degree 3) MSE: 0.1630


In [99]:
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [157]:
X = df1.loc[:, IVs]
y = df1['cabinet_proportion']  

In [158]:
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [172]:

# fit model no training data
xgmodel = XGBRegressor()
xgmodel.fit(X_train, y_train)
# make predictions for test data
y_pred = xgmodel.predict(X_test)

mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R² score

r2_train = r2_score(y_train, xgmodel.predict(X_train))

# Print results
print("Mean Squared Error: %.10f" % mse)
print("R² Score: %.10f" % r2)

print(r2_train)
y_pred.shape


Mean Squared Error: 0.0033684862
R² Score: 0.9277036810
0.9999132029743626


(132,)

In [160]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 

In [161]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [162]:
dmodel = DecisionTreeRegressor(max_depth=3)  # Adjust max_depth to control complexity
dmodel.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=3)

In [163]:
y_pred = dmodel.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.005506028015984318


In [164]:
from sklearn.ensemble import RandomForestRegressor

In [165]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.002876733968714926
